<a href="https://colab.research.google.com/github/jackliuuu/image_detection/blob/master/model_test_shopee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "tensorflow_hub>=0.6.0"

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.utils import to_categorical

from tqdm import tqdm
import os
import glob
import cv2

In [ ]:
train_dir='/content/drive/My Drive/shopee-product-detection-dataset/train/train'
test_dir='/content/drive/My Drive/shopee-product-detection-dataset/test'

In [ ]:
batch_size = 2048
epochs = 10
IMG_HEIGHT = 128
IMG_WIDTH = 128

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1 / 255.0)

In [ ]:
batch_size = 128
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
)
valid_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='validation',
    shuffle=True
)
test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=128,
    class_mode='categorical',
    shuffle=False)

In [ ]:
sample_training_images, _ = next(train_generator)

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:5])

In [ ]:
base_model = tf.keras.applications.InceptionV3(input_shape=(128,128,3),
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
feature_batch = base_model(sample_training_images)
print(sample_training_images.shape)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
m = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(42,activation='softmax')
])

In [ ]:
base_model.trainable = True


In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
m.summary()

In [ ]:
m.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

In [ ]:
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      m.optimizer.lr.numpy()))
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/ckpt/',
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [ ]:
history=m.fit(train_generator,
              steps_per_epoch=10,
              epochs=100,
              validation_data=valid_generator,
              #callbacks=[callbacks]
              )
m.save('my_model.h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model=tf.keras.models.load_model('/content/drive/My Drive/my_model.h5')

In [ ]:
pred=model.predict_generator(test_generator)

In [ ]:
pred

In [ ]:
predicted_class_indices = np.argmax(pred, axis=1)

In [ ]:
labels = (train_generator.class_indices)
label = dict((v,k) for k,v in labels.items())

In [ ]:
predictions = [label[i] for i in predicted_class_indices]

In [ ]:
predictions

In [ ]:
dict_new_output={}
filenames = test_generator.filenames
for idx in range(len(filenames )):
    print('predict  ' + predictions[idx])
    if '(' not in filenames[idx][5:] :
      dict_new_output.update({filenames[idx][5:]:predictions[idx]})
      print('title    %s' % filenames[idx][5:])
    print('')

In [ ]:
dict_new_output['fe19ad2018c43627eaeda2f82c1abbf2.jpg']

In [ ]:
test_csv=pd.read_csv('/content/drive/My Drive/shopee-product-detection-dataset/test.csv')

In [ ]:
test_csv.head()

In [ ]:
for i in range(len(test_csv['filename'])):
  test_csv['category'][i]=dict_new_output[test_csv['filename'][i]]
test_csv['category'] = test_csv.category.apply(lambda c: str(c).zfill(2))

In [ ]:
test_csv.head()

In [ ]:
test_csv.to_csv('submission.csv',index=False)